In [6]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from spark_util import save_rdd, get_parser

import json, re, codecs, sys, HTMLParser


JSON_PATTERN = re.compile(r'^\s*<text xml:space="preserve">(\{&quot;type&quot;:&quot;item&quot;,&quot;id&quot;:&quot;Q.*&quot;,&quot;labels&quot;:.*)</text>')
WIKI_PATTERN = re.compile(r'^(.*)wiki$')
PARSER = HTMLParser.HTMLParser()

def get_WILL(line):
    match = JSON_PATTERN.match(line)
    if not match:
        return []
    line = PARSER.unescape(match.group(1))
    obj = json.loads(line)
    links = obj['sitelinks']
    if len(links) == 0:
      links = dict()
    ret = []
    for wiki in sorted(links.keys()):
        m = WIKI_PATTERN.match(wiki)
        if m:
            lang = m.group(1)
            ret.append('\t'.join([obj['id'], lang, links[wiki]['title']]))
    return ret


In [7]:
dumpfile = '/user/ellery/translation-recs-app/data/wikidata/wikidatawiki-latest-pages-articles.xml.bz2'
dump = sc.textFile(dumpfile)

In [8]:
dump.flatMap(get_WILL).take(10)

[u'Q15\tace\tAfrika',
 u'Q15\taf\tAfrika',
 u'Q15\tak\tAfrika',
 u'Q15\tals\tAfrika',
 u'Q15\tam\t\u12a0\u134d\u122a\u1243',
 u'Q15\tang\tAffrica',
 u'Q15\tan\tAfrica',
 u'Q15\tarc\t\u0710\u0726\u072a\u071d\u0729\u0710',
 u'Q15\tar\t\u0623\u0641\u0631\u064a\u0642\u064a\u0627',
 u'Q15\tarz\t\u0627\u0641\u0631\u064a\u0642\u064a\u0627']

In [ ]:
WILLfile = '/user/ellery/translation-recs-app/data/wikidata/WILLs.tsv'
os.system('hadoop fs -rm -r ' + WILLfile)

In [ ]:
dump.flatMap(get_WILL).saveAsTextFile ( WILLfile)